In [1]:
import numpy as np
import torch
import pickle as pkl
from tqdm import tqdm
from generic_model import *
from utils import *
from matplotlib import pyplot as plt

In [2]:
device = 'cuda' if torch.cuda.is_available() else ('cpu')
device

'cuda'

In [3]:
for dataset in ['8_10000_25','16_10000_15']:
    for distance, iterated in [(False,False), (True,False)]:
        for supervised in [True, False]:
            results_path = f'results/generic_training_results_{dataset_string(dataset,distance,iterated)}{"" if supervised else "_unsupervised"}.pkl'
            print(results_path)
            if not os.path.isfile(results_path):
                print('loading data')
                with open(f'../data/graphs_{dataset}.pkl','rb') as f:
                    graphs = pkl.load(f)
                with open(f'../data/mercer_{dataset_string(dataset,distance,iterated)}.pkl','rb') as f:
                    mercer_pos,mercer_neg = pkl.load(f)
                with open(f'../data/{"distance" if distance else "kernel"}_matrix_{dataset}.pkl','rb') as f:
                    kernel_matrix = pkl.load(f)
                
                print('preparing data')
                targets = np.concatenate([mercer_pos,mercer_neg],axis=-1)
                targets = torch.tensor(targets.squeeze(1),dtype=torch.float32).to(device)
                data = torch.tensor(np.stack([np.asarray(graph.todense()) for graph in graphs],axis=0),dtype=torch.float32).to(device)
                kernel_matrix = torch.tensor(kernel_matrix,dtype=torch.float32).to(device)
                print('initializing model')
                model = GenericModel(size=graphs[0].shape[0], embedding_size=targets.shape[1],device=device)
                print('model parameters:',sum(p.numel() for p in model.parameters()))
                # model = model.to(device)
                # data = model.prepare_data(data)
                if supervised:
                    results, test_data, test_kernel = fit_supervised(model,data,targets,kernel_matrix,mercer_pos.shape[-1],return_test=True)
                else:
                    results, test_data, test_kernel  = fit_unsupervised(model,data,kernel_matrix,return_test=True)
                with open(results_path,'wb') as f:
                    pkl.dump((results, test_data, test_kernel), f)
                torch.save(model,f'models/model_{dataset_string(dataset,distance,iterated)}{"" if supervised else "_unsupervised"}.pt')
            else:
                with open(f'../data/mercer_{dataset_string(dataset,distance,iterated)}.pkl','rb') as f:
                    mercer_pos,mercer_neg = pkl.load(f)
                with open(results_path,'rb') as f:
                    results, test_data, test_kernel = pkl.load(f)
                model = torch.load(f'models/model_{dataset_string(dataset,distance,iterated)}{"" if supervised else "_unsupervised"}.pt')
                print('model parameters:',sum(p.numel() for p in model.parameters()))
                
            epoch, loss, test_loss, r_squared = zip(*results)
            
            plt.figure(figsize=(12,8))
            plt.rcParams.update({'font.size': 13})
            
            cmap = plt.cm.magma  # define the colormap
            cmaplist = [cmap(i) for i in range(cmap.N)] # extract all colors from the colormap
            
            rho = "$\\delta$"
            plt.title(f'Loss per epoch - generic model, {"" if supervised else "un"}supervised, for {rho if distance else "$K$"} on {DATASET_MAP[dataset]}')
            plt.gca().set_ylabel('loss')
            plt.gca().set_xlabel(f'epoch')
            
            plt.plot(epoch,loss,label='loss', lw=3, color=cmaplist[64])
            plt.plot(epoch,test_loss,label='test loss',lw=3,color=cmaplist[196])
            
            plt.tight_layout()
            plt.legend()
            plt.gca().set_ylim(bottom=.5*min(min(loss),min(test_loss)),top=1.25*loss[100])
            plt.savefig(f'../plots/loss_{dataset_string(dataset,distance,iterated)}_{"" if supervised else "un"}supervised.png',format='png',dpi=300)
            # plt.show()
            plt.close()
            
            
            plt.figure(figsize=(12,8))
            plt.rcParams.update({'font.size': 13})
            
            cmap = plt.cm.magma  # define the colormap
            cmaplist = [cmap(i) for i in range(cmap.N)] # extract all colors from the colormap
            
            rho = "$\\delta$"
            plt.title(f'Test sample $R^2$ per epoch - generic model, {"" if supervised else "un"}supervised, for {rho if distance else "$K$"} on {DATASET_MAP[dataset]}')
            plt.gca().set_ylabel('$R^2$')
            plt.gca().set_xlabel(f'epoch')
            
            plt.plot(epoch,r_squared,label='$R^2$',lw=3, color=cmaplist[128])
            plt.tight_layout()
            plt.legend()
            plt.gca().set_ylim(bottom=max(0,r_squared[999]-.1),top=1.)
            plt.savefig(f'../plots/r_squared_{dataset_string(dataset,distance,iterated)}_{"" if supervised else "un"}supervised.png',format='png',dpi=300)
            # plt.show()
            plt.close()
            
            predictions = model(test_data).to('cpu')
            n_positive = mercer_pos.shape[-1] if supervised else predictions.shape[-1]//2
            print(f'{n_positive=}')
            predictions_pos, predictions_neg = predictions[:,:n_positive], predictions[:,n_positive:]
            predicted_kernel = predictions_pos @ predictions_pos.T - predictions_neg @ predictions_neg.T
            
            indices = torch.triu_indices(*predicted_kernel.shape, offset=1)
            subsample = np.random.choice(np.arange(len(indices[0])), 10_000,replace=False)
            indices = indices[:,subsample]
            predicted_kernel_vals = predicted_kernel[indices[0],indices[1]]
            kernel_vals = test_kernel.to('cpu')[indices[0],indices[1]]
            
            plt.figure(figsize=(12,8))
            plt.rcParams.update({'font.size': 13})
            
            cmap = plt.cm.magma  # define the colormap
            cmaplist = [cmap(i) for i in range(cmap.N)] # extract all colors from the colormap
            
            plt.title(f'Predictions vs. true kernel values for the test sample - generic model, {"" if supervised else "un"}supervised, for {rho if distance else "$K$"} on {DATASET_MAP[dataset]}')
            plt.gca().set_ylabel('predicted')
            plt.gca().set_xlabel(f'true kernel value')
            
            plt.scatter(kernel_vals.detach().numpy(),predicted_kernel_vals.detach().numpy(),s=2,alpha=.2,color=cmaplist[192])
            plt.plot([0,kernel_vals.detach().numpy().max()],[0,kernel_vals.detach().numpy().max()],color=cmaplist[0])
            plt.tight_layout()
            
            plt.savefig(f'../plots/scatterplot_{dataset_string(dataset,distance,iterated)}_{"" if supervised else "un"}supervised.png',format='png',dpi=300)
            # plt.show()
            plt.close()
            
            print('min test loss:', min(test_loss))
            print('max R^2:', max(r_squared))
            

results/generic_training_results_8_10000_25.pkl


FileNotFoundError: [Errno 2] No such file or directory: 'model_8_10000_25.pt'